In [2]:
import filecmp
import zipfile
import os
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd
import branca.colormap as cm
%load_ext streamlit


The streamlit module is not an IPython extension.


### Unzip and rename the files
Each zip_file have two files: 
- One with the names of the neighborhoods
- One with the rent and sale values for those six months

So, one year has four files

In [3]:
zip_file_path = './zipped_files'
unzipped_file_path = './unzipped_files'

if not os.path.exists(unzipped_file_path):
    for file in os.listdir(zip_file_path):
        zip_ref = zipfile.ZipFile(os.path.join(zip_file_path, file), 'r')
        zip_ref.extractall(unzipped_file_path)
        zip_ref.close()



In [4]:
# Check if a the first file in the unzipped folder is with has number in the first letter of the file name

if not os.listdir(unzipped_file_path)[0][0].isdigit():
    for file in os.listdir(unzipped_file_path):
        if file.endswith('.csv'):
            parts = file.split('_')
            new_parts = parts[3:]
            new_parts[0] = new_parts[0][:4] + '_' + new_parts[0][4:]
            new_file = '_'.join(new_parts)

            os.rename(os.path.join(unzipped_file_path, file),
                      os.path.join(unzipped_file_path, new_file))


IndexError: list index out of range

### Clean the structures and the file names

First of all we need to check if the neighborhoods files are always the same, and if so, we'll just consider one and call it NEIGHBORHOODS.csv and rename the rest with VALUES_year_1.csv or VALUES_year_2.csv

In [ ]:
if len(os.listdir(unzipped_file_path)) > 22:
    zone_files = [f for f in os.listdir(unzipped_file_path) if 'ZONE' in f]
    zone_files.sort()

    # Check if the Zone files are the same
    first_file = os.path.join(unzipped_file_path, zone_files[0])
    
    ## Exclude the first line of the file
    with open(first_file, 'r') as f:
        first_file_lines = f.readlines()[1:]
    
    for file in zone_files[1:]:
        other_zone_file = os.path.join(unzipped_file_path, file)
        
        ## Exclude the first line of the file
        with open(other_zone_file, 'r') as f:
            other_file_lines = f.readlines()[1:]

        are_equal = first_file_lines == other_file_lines
    
        if not are_equal:
            print(f'The files are not all the same')
            break
    if are_equal:
        print('All the files are the same')

The files are not all the same


Unfortunately the files with the neighbors names are **not** all the same. Let's check if they also change between semesters of the same year

In [ ]:
if len(os.listdir(unzipped_file_path)) > 22:
    checker = 1
    # Compare the 'zone' files of the same year
    for index, file in enumerate(zone_files):
        if checker != 0:
            other_zone_file = os.path.join(
                unzipped_file_path, zone_files[index + 1])

            # Exclude the first line of the file
            with open(os.path.join(unzipped_file_path, file), 'r') as f1, open(os.path.join(unzipped_file_path, zone_files[index + 1]), 'r') as f2:
                file1_lines = f1.readlines()[1:]
                file2_lines = f2.readlines()[1:]

            are_equal = file1_lines == file2_lines

            if not are_equal:
                print(
                    f'\n The files are not all the same')
                break
            checker = 0
        else:
            continue

    if are_equal:
        print('All the files are the same')


All the files are the same


The neighborhoods file stay the same between semester, so we will remove one

In [ ]:
for file_name in zone_files:
    if file_name.endswith('_2_ZONE.csv'):
        file = os.path.join(unzipped_file_path, file_name)
        os.remove(file)

for file_name in zone_files:
    if file_name.endswith('_1_ZONE.csv'):
        # Rename the file keeping the year
        year = file_name[:4]

        os.rename(os.path.join(unzipped_file_path, file_name),
                    os.path.join(unzipped_file_path, f'{year}_ZONE.csv'))

### Open all the Rent/Sale files in DataFrames and merge them together in one
**We will keep only the neighborhoods present in all of them**



In [ ]:
# Create a function that given the OMI files returns one dataframe
def create_dataframe_from_OMI_files(neighborhoods_file, values_file1, values_file2):
    neighborhoods = pd.read_csv(neighborhoods_file, sep=';', skiprows=1)
    values1 = pd.read_csv(values_file1, sep=';', skiprows=1)
    values2 = pd.read_csv(values_file2, sep=';', skiprows=1)
    year = values_file1.split('/')[2].split('_')[0]

    # Drop columns that are not needed
    neighborhoods.drop(columns=['Area_territoriale', 'Regione', 'Prov', 'Comune_ISTAT', 'Comune_cat', 'Sez', 'Comune_amm',
                                'Comune_descrizione', 'Fascia', 'LinkZona', 'Cod_tip_prev', 'Stato_prev', 'Microzona'], inplace=True, errors='ignore')
    values1.drop(columns=['Area_territoriale', 'Regione', 'Prov', 'Comune_ISTAT', 'Comune_cat', 'Sez', 'Comune_amm',
                          'Comune_descrizione', 'Fascia', 'LinkZona', 'Cod_Tip', 'Stato_prev', 'Sup_NL_compr', 'Sup_NL_loc'], inplace=True, errors='ignore')
    values2.drop(columns=['Area_territoriale', 'Regione', 'Prov', 'Comune_ISTAT', 'Comune_cat', 'Sez', 'Comune_amm',
                          'Comune_descrizione', 'Fascia', 'LinkZona', 'Cod_Tip', 'Stato_prev', 'Sup_NL_compr', 'Sup_NL_loc'], inplace=True, errors='ignore')

    # Add columns to the values files regarding the semester
    values1['Semestre'] = 1
    values2['Semestre'] = 2
    
    values = pd.concat([values1, values2])

    neighborhoods_values = pd.merge(neighborhoods, values, on='Zona')
    neighborhoods_values.drop(columns=['Zona'], inplace=True)
    neighborhoods_values['Anno'] = year

    # Drop unnamed columns
    neighborhoods_values.drop(columns=[
                              col for col in neighborhoods_values.columns if 'Unnamed' in col], inplace=True)

    neighborhoods_values.drop(columns=['Area_territoriale', 'Regione', 'Prov', 'Comune_ISTAT', 'Comune_cat', 'Sez', 'Comune_amm',
                                       'Comune_descrizione', 'Fascia', 'LinkZona', 'Cod_tip_prev', 'Stato_prev', 'Microzona',
                                       'Area_territoriale', 'Regione', 'Prov', 'Comune_ISTAT', 'Comune_cat', 'Sez', 'Comune_amm',
                                       'Comune_descrizione', 'Fascia', 'LinkZona', 'Cod_Tip', 'Stato_prev', 'Sup_NL_compr', 'Sup_NL_loc'], inplace=True, errors='ignore')

    return neighborhoods_values


# create_dataframe_from_OMI_files(os.path.join(unzipped_file_path, '2013_ZONE.csv'), os.path.join(unzipped_file_path, '2013_1_VALORI.csv'), os.path.join(unzipped_file_path, '2013_2_VALORI.csv'))



In [ ]:
checker = 0
csvs_separated_by_year_path = './merged_csvs'
os.mkdir(csvs_separated_by_year_path) if not os.path.exists(
    csvs_separated_by_year_path) else None


if len(os.listdir(csvs_separated_by_year_path)) < 1:
    # Sort the files by year
    csv_files = sorted([f for f in os.listdir(
        unzipped_file_path) if f.endswith('.csv')])

    for index, file in enumerate(csv_files):
        year = file.split('_')[0]

        checker = 1 if year not in csv_files[index + 2] else 0

        if checker == 1:
            continue
        else:
            values_file1 = os.path.join(
                unzipped_file_path, file)

            values_file2 = os.path.join(
                unzipped_file_path, csv_files[index + 1])

            neighborhoods_file = os.path.join(
                unzipped_file_path, csv_files[index + 2])

            df = create_dataframe_from_OMI_files(
                neighborhoods_file, values_file1, values_file2)
            df.reset_index(drop=True, inplace=True)
            df.to_csv(
                os.path.join(os.getcwd(), csvs_separated_by_year_path, year + '_values.csv'), index=False, columns=[
                    'Zona_Descr', 'Descr_Tipologia', 'Stato', 'Compr_min', 'Compr_max', 'Loc_min', 'Loc_max', 'Semestre', 'Anno'])

        if year == '2022':
            break


Before merging all of the different years in one file, we will have to make a row for each neighborhood, because at the moment neighborhood with the same values are displayed on the same line and are grouped differently in each dataset

In [ ]:

for file in os.listdir(csvs_separated_by_year_path):
    df = pd.read_csv(os.path.join(csvs_separated_by_year_path, file))

    df['Zona_Descr'] = df['Zona_Descr'].str.strip("-'")
    # Split the Zona_Descr column by comma and explode it to create a new row for each zone
    df = df.assign(Zona_Descr=df['Zona_Descr'].str.split(", ")).explode('Zona_Descr')
    df = df.reset_index(drop=True)

    # Save the file as a csv overwriting the old one
    df.to_csv(os.path.join(csvs_separated_by_year_path, file), index=False)


# Create a function that given the OMI files returns one dataframe
def concat_csvs(csv_files):
    dfs = []
    for file in csv_files:
        df = pd.read_csv(os.path.join(csvs_separated_by_year_path, file))
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)
    return concatenated_df

### Merge all the csv files of the neighborhoods sale and rent values in one DataFrame

In [ ]:
nei_sale_rent = concat_csvs(os.listdir(csvs_separated_by_year_path))
nei_sale_rent.rename(columns={'Zona_Descr': 'Neighborhood', 'Descr_Tipologia': 'Type', 'Stato': 'Status', 'Compr_min': 'Min_Sale_Price',
                     'Compr_max': 'Max_Sale_Price', 'Loc_min': 'Min_Rent_Price', 'Loc_max': 'Max_Rent_Price', 'Semestre': 'Semester', 'Anno': 'Year'}, inplace=True)

# Change the types of the columns in float and int
nei_sale_rent['Min_Rent_Price'] = nei_sale_rent['Min_Rent_Price'].astype(str).str.replace(',', '.').astype(float)
nei_sale_rent['Max_Rent_Price'] = nei_sale_rent['Max_Rent_Price'].astype(str).str.replace(',', '.').astype(float)
nei_sale_rent['Min_Sale_Price'] = nei_sale_rent['Min_Sale_Price'].astype(int)
nei_sale_rent['Max_Sale_Price'] = nei_sale_rent['Max_Sale_Price'].astype(int)
nei_sale_rent['Semester'] = nei_sale_rent['Semester'].astype(int)
nei_sale_rent['Year'] = nei_sale_rent['Year'].astype(int)

# Make neighborhoods without a comma
nei_sale_rent['Neighborhood'] = nei_sale_rent['Neighborhood'].str.replace(',', '')
# Check if neighborhoods have - and take the second part of the string
nei_sale_rent['Neighborhood'] = nei_sale_rent['Neighborhood'].apply(lambda x: x.split('-')[1] if '-' in x else x)
# Make CNA MERLATA A and C.NA MERLATA neighborhoods the same in CAMERLATA
nei_sale_rent['Neighborhood'] = nei_sale_rent['Neighborhood'].apply(lambda x: 'CAMERLATA' if 'C.NA MERLATA' in x else x)
nei_sale_rent['Neighborhood'] = nei_sale_rent['Neighborhood'].apply(lambda x: 'CAMERLATA' if 'CNA MERLATA' in x else x)
nei_sale_rent['Neighborhood'] = nei_sale_rent['Neighborhood'].apply(lambda x: 'SAN CARLO' if 'SAN CARLO B.' in x else x)
# Remove spaces from neighborhoods
nei_sale_rent['Neighborhood'] = nei_sale_rent['Neighborhood'].str.strip()
# Add the average sale and rent price 
nei_sale_rent['Avg_Sale_Price'] = (
    nei_sale_rent['Min_Sale_Price'] + nei_sale_rent['Max_Sale_Price']) / 2
nei_sale_rent['Avg_Rent_Price'] = (
    nei_sale_rent['Min_Rent_Price'] + nei_sale_rent['Max_Rent_Price']) / 2

nei_sale_rent.to_csv(os.path.join(os.getcwd(), 'rent_sale_per_neighborhood.csv'), index=False)

In [ ]:
import difflib


def modify_array(array1, array2):
    modified_array2 = []
    
    for item2 in array2:
        closest_match = difflib.get_close_matches(item2, array1, n=1, cutoff=0.3)
        if closest_match:
            modified_array2.append(closest_match[0])
        else:
            modified_array2.append(item2)
    
    return modified_array2

## Modify the geojson file 

In [ ]:
# Import geojson
neighborhood = gpd.GeoDataFrame.from_file(
    './milan_districts.geojson')

# Check which neighborhoods are in the geojson file but not in the OMI files
neighborhoods_in_geojson = neighborhood['NIL'].unique()
neighborhoods_in_omi = nei_sale_rent['Neighborhood'].unique()

print('Neighborhoods in geojson but not in OMI files: ', set(
    neighborhoods_in_geojson) - set(neighborhoods_in_omi))

# Modify NIL column in the geojson file with the function modify_array
neighborhood['NIL'] = modify_array(neighborhoods_in_omi, neighborhoods_in_geojson)
print(neighborhood['NIL'].unique())

# Save the new geojson file
neighborhood.to_file('./milan_districts_modified.geojson', driver='GeoJSON')



Neighborhoods in geojson but not in OMI files:  {'QUARTO CAGNINO', 'DERGANO', 'TALIEDO - MORSENCHIO - Q.RE FORLANINI', 'SAN SIRO', 'ISOLA', 'QT 8', 'GUASTALLA', 'MAGGIORE - MUSOCCO - CERTOSA', 'DE ANGELI - MONTE ROSA', 'TRE TORRI', 'GRATOSOGLIO - Q.RE MISSAGLIA - Q.RE TERRAZZE', 'PORTA VIGENTINA - PORTA LODOVICA', "MONLUE' - PONTE LAMBRO", 'TIBALDI', 'STADERA - CHIESA ROSSA - Q.RE TORRETTA - CONCA FALLATA', 'BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA', "PARCO BOSCO IN CITTA'", 'FORZE ARMATE', 'CASCINA MERLATA', 'PTA ROMANA', 'BANDE NERE', 'ASSIANO', 'SCALO ROMANA', 'STADIO - IPPODROMI', 'CIMIANO - ROTTOLE - Q.RE FELTRE', 'BRUZZANO', 'GRECO - SEGNANO', 'MONCUCCO - SAN CRISTOFORO', 'STAZIONE CENTRALE - PONTE SEVESO', 'PADOVA - TURRO - CRESCENZAGO', 'TRENNO', 'PARCO DEI NAVIGLI', 'PORTA GARIBALDI - PORTA NUOVA', 'MAGENTA - S. VITTORE', 'PORTA MAGENTA', 'GHISOLFA', 'Q.RE GALLARATESE - Q.RE SAN LEONARDO - LAMPUGNANO', 'LAMBRATE - ORTICA', 'LODI - CORVETTO', 'PARCO DELLE ABBAZIE', 'BUENOS AIRE

### Fix Mispellings

In [19]:
airbnb = pd.read_csv('./airbnb_listings2022.csv')

# Fix mispelled column names
corrections = {
    'neighbourhood': 'neighborhood',
    'neighbourhood_cleansed': 'neighborhood_cleansed',
    'neighbourhood_group_cleansed': 'neighborhood_group_cleansed',
}

airbnb.rename(columns=corrections, inplace=True)
airbnb.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighborhood',
       'neighborhood_cleansed', 'neighborhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calen

In [48]:
airbnb['availability_365'].unique()

array([358, 363,  88, 173,   0, 184, 219, 327, 203,  78, 359,  98,  52,
       163,  55, 355, 177, 239, 204,  67, 112, 364, 297,  21,  14, 362,
       268, 246, 349, 350, 322, 365, 346, 142, 192, 299,   1, 138, 345,
       151,  68,  90, 225, 348,  36, 179, 123,  81, 220, 174, 205, 157,
       166, 330,  95,  20, 224, 356,   9, 263,  79,  82,  28, 242, 172,
       334,  18, 354, 258, 303, 296, 264, 212, 295,  87, 245,  37,  76,
       180, 160,  97, 120, 159,  13, 276,  26, 227, 357, 352, 314, 312,
       342,  50, 269, 234, 206, 215, 213, 185,  42,   6,  94, 110, 241,
       320, 315,  93, 272, 158, 101, 323, 178, 265, 351,  89,  19, 298,
       240, 347,  84, 317, 169,  11,  47, 235, 168, 164, 201,  32, 236,
        35,  71, 293, 286, 171,   8, 167, 217,  85, 232,  59, 301,  83,
        54, 156, 343, 332,  99,  65, 183,  45,  12, 360, 339, 146, 108,
        64, 319, 161, 337,  40, 200, 153, 150,  66, 300, 111, 139, 221,
       321, 251, 214,   4, 261, 207, 291, 230, 211, 182,   2, 24

In [53]:
# Select only the columns that are useful for the analysis
bnb = airbnb[['id', 'name', 'neighborhood_cleansed', 'latitude', 'longitude', 'host_listings_count', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'price', 'availability_365', 'review_scores_rating']]
# Save the file
bnb.to_csv('./airbnb_listings2022_modified.csv', index=False)
